In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.metrics import *
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[2], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='selu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='selu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation= 'selu' ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [9]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [11]:
run_experiment(X,y)

Epoch 1/10
29/29 [==============================] - 1s 19ms/step - loss: 38.4072 - accuracy: 0.6221 - precision: 0.6056 - recall: 0.5561 - auc: 0.6222
Epoch 2/10
29/29 [==============================] - 1s 18ms/step - loss: 0.5807 - accuracy: 0.8916 - precision: 0.8834 - recall: 0.8855 - auc: 0.9467
Epoch 3/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0784 - accuracy: 0.9726 - precision: 0.9611 - recall: 0.9813 - auc: 0.9952
Epoch 4/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0193 - accuracy: 0.9923 - precision: 0.9930 - recall: 0.9907 - auc: 0.9998
Epoch 5/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0102 - accuracy: 0.9945 - precision: 0.9930 - recall: 0.9953 - auc: 1.0000
Epoch 6/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0351 - accuracy: 0.9934 - precision: 0.9976 - recall: 0.9883 - auc: 0.9987
Epoch 7/10
29/29 [==============================] - 0s 17ms/step - loss: 0.0113 - accuracy: 0.9956 

In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='selu', input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64, kernel_size=3, activation='selu'))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64, kernel_size=3, activation='selu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation= 'selu' ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [13]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [14]:
run_experiment_2(X,y)

Epoch 1/10
29/29 [==============================] - 1s 24ms/step - loss: 91.8728 - accuracy: 0.5433 - precision_5: 0.5144 - recall_5: 0.4603 - auc_5: 0.5426
Epoch 2/10
29/29 [==============================] - 1s 23ms/step - loss: 2.2839 - accuracy: 0.6637 - precision_5: 0.6274 - recall_5: 0.6963 - auc_5: 0.7124
Epoch 3/10
29/29 [==============================] - 1s 23ms/step - loss: 0.6204 - accuracy: 0.6889 - precision_5: 0.6475 - recall_5: 0.7383 - auc_5: 0.7520
Epoch 4/10
29/29 [==============================] - 1s 23ms/step - loss: 0.5951 - accuracy: 0.7054 - precision_5: 0.6871 - recall_5: 0.6822 - auc_5: 0.7912
Epoch 5/10
29/29 [==============================] - 1s 23ms/step - loss: 0.5082 - accuracy: 0.7481 - precision_5: 0.7281 - recall_5: 0.7383 - auc_5: 0.8387
Epoch 6/10
29/29 [==============================] - 1s 23ms/step - loss: 0.6425 - accuracy: 0.7689 - precision_5: 0.7374 - recall_5: 0.7874 - auc_5: 0.8555
Epoch 7/10
29/29 [==============================] - 1s 23ms/ste

In [15]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='selu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='selu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation= 'selu' ))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation= 'selu' ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [16]:
def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_3(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [17]:
run_experiment_3(X,y)

Epoch 1/10
29/29 [==============================] - 1s 18ms/step - loss: 22.0066 - accuracy: 0.6539 - precision_10: 0.6407 - recall_10: 0.5958 - auc_10: 0.6579
Epoch 2/10
29/29 [==============================] - 1s 19ms/step - loss: 0.9959 - accuracy: 0.9135 - precision_10: 0.9145 - recall_10: 0.8995 - auc_10: 0.9353
Epoch 3/10
29/29 [==============================] - 1s 18ms/step - loss: 0.8312 - accuracy: 0.9354 - precision_10: 0.9321 - recall_10: 0.9299 - auc_10: 0.9574
Epoch 4/10
29/29 [==============================] - 1s 18ms/step - loss: 0.7565 - accuracy: 0.9365 - precision_10: 0.9243 - recall_10: 0.9416 - auc_10: 0.9568
Epoch 5/10
29/29 [==============================] - 1s 18ms/step - loss: 0.3200 - accuracy: 0.9595 - precision_10: 0.9536 - recall_10: 0.9603 - auc_10: 0.9777
Epoch 6/10
29/29 [==============================] - 1s 18ms/step - loss: 0.3916 - accuracy: 0.9715 - precision_10: 0.9741 - recall_10: 0.9650 - auc_10: 0.9856
Epoch 7/10
29/29 [===========================